In [1]:
import sys
sys.path.append("..")

from glob import glob
import matplotlib.pyplot as plt
import ipywidgets as ipw
from IPython.display import Audio
import numpy as np
import pickle
import yaml

from lib.notebooks import plot_groups_metrics
from imitative_agent import ImitativeAgent

current path: /mnt/c/Users/vpaul/OneDrive - CentraleSupelec/Inner_Speech/agent/imitative_agent_inner_speech_silent_finetune


In [6]:
agents_path = glob("../out/imitative_agent_inner_speech_silent_finetune_nojerk7/*/")
agents_path.sort()
print(f"Found {len(agents_path)} agents")


Found 30 agents


In [ ]:
# Dictionary to store metrics for different groups of agents
groups_metrics = {}

# Dictionary to store various parameters and performance metrics for each agent
agents_loss = {
    "path": [],                        # Path to agent directory
    "datasets": [],                    # Training datasets used
    "inverse_learning_rate": [],       # Learning rate for inverse model
    "inverse_layers": [],             # Layer configuration of inverse model
    "inverse_dropout_p": [],          # Dropout probability for inverse model
    "direct_learning_rate": [],       # Learning rate for direct model
    "direct_layers": [],              # Layer configuration of direct model
    "direct_dropout_p": [],           # Dropout probability for direct model
    "jerk_weight": [],               # Weight of jerk loss term
    "direct_estimation_error": [],    # Test error of direct model
    "inverse_estimation_error": [],   # Test error of inverse model
    "jerk": [],                      # Jerk metric on test set
    "repetition_error": [],          # Repetition error on test set
}

# Iterate through all agent directories to collect metrics and parameters
for agent_path in agents_path:
    # Load agent configuration without neural networks for efficiency
    agent = CommunicativeAgent.reload(agent_path, load_nn=False)
    config = agent.config
    
    # Load stored metrics from training
    with open("%s/metrics.pickle" % agent_path, "rb") as f:
        metrics = pickle.load(f)
    
    # Store basic agent information
    agents_loss["path"].append(agent_path[-5:-1])
    agents_loss["datasets"].append(",".join(agent.sound_quantizer.config['dataset']['names']))

    # Store inverse model parameters
    agents_loss["inverse_learning_rate"].append(config['training']['inverse_model_learning_rate'])
    agents_loss["inverse_layers"].append(f"{config['model']['inverse_model']['num_layers']}x{config['model']['inverse_model']['hidden_size']}")
    agents_loss["inverse_dropout_p"].append(config['model']['inverse_model']['dropout_p'])

    # Store direct model parameters, handling cases where synthesizer is used as direct model
    if 'use_synth_as_direct_model' not in config['model']:
        agents_loss["direct_learning_rate"].append(config['training']['direct_model_learning_rate'])
        agents_loss["direct_layers"].append(f"{len(config['model']['direct_model']['hidden_layers'])}x{config['model']['direct_model']['hidden_layers'][0]}")
        agents_loss["direct_dropout_p"].append(config['model']['direct_model']['dropout_p'])
    else:
        # Use placeholder values when synthesizer is used as direct model
        agents_loss["direct_learning_rate"].append(0)
        agents_loss["direct_layers"].append("synth")
        agents_loss["direct_dropout_p"].append(0)
    
    agents_loss["jerk_weight"].append(config['training']['jerk_loss_weight'])

    # Find index of best validation performance for reporting test metrics
    final_loss_index = np.argmin(metrics["validation"]["inverse_model_repetition_error"])
    
    # Store test performance metrics
    if 'use_synth_as_direct_model' not in config['model']:
        agents_loss["direct_estimation_error"].append(metrics["test"]["direct_model_estimation_error"][final_loss_index])
    else:
        agents_loss["direct_estimation_error"].append(0)

    agents_loss["inverse_estimation_error"].append(metrics["test"]["inverse_model_estimation_error"][final_loss_index])
    agents_loss["jerk"].append(metrics["test"]["inverse_model_jerk"][final_loss_index])
    agents_loss["repetition_error"].append(metrics["test"]["inverse_model_repetition_error"][final_loss_index])
    
    # Create group name based on key configuration parameters
    group_name = "\n".join((
        f"datasets={','.join(agent.sound_quantizer.config['dataset']['names'])}",
        f"synth_art={agent.synthesizer.config['dataset']['art_type']}",
        f"jerk_w={config['training']['jerk_loss_weight']}",
        # f"frame_padding={config['model']['sound_quantizer']['frame_padding']}",
    ))
    
    # Store metrics for this group
    if group_name not in groups_metrics:
        groups_metrics[group_name] = {}
    groups_metrics[group_name][agent_path] = metrics

# Convert collected data to pandas DataFrame for easier analysis
agents_loss = pd.DataFrame(agents_loss)

In [7]:
# Dictionary to store metrics for different groups of agents
groups_metrics = {}


for agent_path in agents_path:
    agent = ImitativeAgent.reload(agent_path, load_nn=False)
    config = agent.config
    with open("%s/metrics.pickle" % agent_path, "rb") as f:
        metrics = pickle.load(f)
    
    if config['training']['jerk_loss_weight'] != 1:
        continue
    
    group_name = "\n".join((
        f"{','.join(config['dataset']['names'])}",
        f"synth_art={agent.synthesizer.config['dataset']['art_type']}",
        f"jerk_c={config['training']['jerk_loss_ceil']}",
        f"jerk_w={config['training']['jerk_loss_weight']}",
        f"bi={config['model']['inverse_model']['bidirectional']}",
    ))
    
    if group_name not in groups_metrics:
        groups_metrics[group_name] = {}
    groups_metrics[group_name][agent_path] = metrics

{'dataset': {'batch_size': 8, 'datasplits_size': [64, 16, 20], 'names': ['pb2007'], 'num_workers': 6, 'shuffle_between_epochs': True, 'sound_type': 'cepstrum'}, 'model': {'direct_model': {'activation': 'relu', 'batch_norm': True, 'dropout_p': 0.25, 'hidden_layers': [256, 256, 256, 256]}, 'inverse_model': {'bidirectional': True, 'dropout_p': 0.25, 'hidden_size': 32, 'num_layers': 2}}, 'synthesizer': {'name': 'ea587b76c95fecef01cfd16c7f5f289d-0/'}, 'training': {'jerk_loss_ceil': 0, 'jerk_loss_weight': 0, 'learning_rate': 0.001, 'max_epochs': 800, 'patience': 100, 'vel_loss_ceil': 0, 'vel_loss_weight': 0}}
{'direct_model': {'activation': 'relu', 'batch_norm': True, 'dropout_p': 0.25, 'hidden_layers': [256, 256, 256, 256]}, 'inverse_model': {'bidirectional': True, 'dropout_p': 0.25, 'hidden_size': 32, 'num_layers': 2}}
{'direct_model': {'activation': 'relu', 'batch_norm': True, 'dropout_p': 0.25, 'hidden_layers': [256, 256, 256, 256]}, 'inverse_model': {'bidirectional': True, 'dropout_p': 

In [8]:
metrics_name = [
    "direct_model_estimation_error",
    "inverse_model_estimation_error",
    "inverse_model_repetition_error",
    "inverse_model_jerk",
]

def show_metrics(split_name="test"):
    plot_groups_metrics(groups_metrics, metrics_name, split_name)
display(ipw.interactive(show_metrics, split_name=["train", "validation", "test"]))

interactive(children=(Dropdown(description='split_name', index=2, options=('train', 'validation', 'test'), val…